# L1: Hierarchical Content Generation

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.<br>
<span style="font-size: larger;">To maintain consistency, the notebooks are run with a 'world state' consistent with the video at the start of each notebook.</span></p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

## Creating a World

In [5]:
system_prompt = f"""
    Seu trabalho é ajudar a criar mundos de fantasia interessantes que os jogadores adorariam explorar.
   
    Instruções:
    Gere apenas texto simples, sem formatação.
    Use uma linguagem simples e clara, sem floreios.
    Cada descrição deve ter no máximo 3-5 frases.
"""

In [6]:

world_prompt = f"""
Gere uma descrição criativa para um mundo de fantasia único.

Saída do conteúdo no formato:
Nome do Mundo: <NOME DO MUNDO>
Descrição do Mundo: <DESCRIÇÃO DO MUNDO>

Nome do Mundo:"""


In [8]:
import google.generativeai as genai
from app_config import LLM_API_KEY,LLM_MODEL

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
    }
genai.configure(api_key=LLM_API_KEY)
client = genai.GenerativeModel(
            model_name=LLM_MODEL,
            generation_config=generation_config,
            system_instruction=system_prompt
            )

output = client.generate_content(world_prompt)

In [10]:
world_output =output.text
print(world_output)

Nome do Mundo: Aethelgard

Descrição do Mundo: Aethelgard é um mundo arquipelágico onde ilhas flutuam em um mar de névoas perenes. As ilhas são de tamanhos variados, desde pequenas ilhotas até grandes continentes, cada uma com sua própria cultura e ecossistema únicos.  A magia é inerente à própria névoa, alimentando a vida e a magia das criaturas que ali habitam.  Construções antigas e misteriosas pontuam a paisagem, guardando segredos perdidos de civilizações passadas.



In [11]:
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('Nome do Mundo: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('Descrição do Mundo:', '').strip()
}

In [12]:
world

{'name': 'Aethelgard',
 'description': 'Aethelgard é um mundo arquipelágico onde ilhas flutuam em um mar de névoas perenes. As ilhas são de tamanhos variados, desde pequenas ilhotas até grandes continentes, cada uma com sua própria cultura e ecossistema únicos.  A magia é inerente à própria névoa, alimentando a vida e a magia das criaturas que ali habitam.  Construções antigas e misteriosas pontuam a paisagem, guardando segredos perdidos de civilizações passadas.'}

## Generating Kingdoms

In [13]:
kingdom_prompt = f"""
Crie 3 reinos diferentes para um mundo de fantasia.
Para cada reino, gere uma descrição com base no mundo em que ele está.
Descreva líderes importantes, culturas e história do reino.

Saída do conteúdo no formato:
Nome do Reino 1: <NOME DO REINO>
Descrição do Reino 1: <DESCRIÇÃO DO REINO>
Nome do Reino 2: <NOME DO REINO>
Descrição do Reino 2: <DESCRIÇÃO DO REINO>
Nome do Reino 3: <NOME DO REINO>
Descrição do Reino 3: <DESCRIÇÃO DO REINO>

Nome do Mundo: {world['name']}
Descrição do Mundo: {world['description']}

Reino 1"""


In [24]:
output = client.generate_content(kingdom_prompt)

In [25]:
kingdoms = {}
kingdoms_output = output.text
kingdoms_output.split('\n\n')

['Nome do Reino 1: Aeridor\nDescrição do Reino 1: Aeridor é uma ilha flutuante extensa, conhecida por suas florestas exuberantes e arquitetura em espiral. Seu governante, a Rainha Lyra, é uma poderosa feiticeira que mantém a paz através de sua sabedoria e magia. A cultura de Aeridor celebra a natureza e a magia, com festivais e rituais dedicados aos espíritos da floresta.  Sua história é marcada por uma era dourada de arte e descoberta, seguida de um período de isolamento que preservou sua cultura única.',
 'Nome do Reino 2: Islas de Pedra\nDescrição do Reino 2: Islas de Pedra é um arquipélago de ilhas rochosas e áridas, com fortalezas construídas em penhascos íngremes.  Governantes poderosos, os Senhores da Pedra, competem por poder e recursos, gerando guerras constantes.  Sua cultura é voltada para a guerra e a sobrevivência, e a sociedade é estratificada e hierárquica. Sua história é repleta de conflitos e conquistas, com reinos emergindo e ruindo ao longo dos séculos.',
 'Nome do R

In [28]:

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom


IndexError: list index out of range

In [ ]:
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

## Generating Towns

In [ ]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""


In [ ]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.completions.create(
      model="meta-llama/Llama-3-70b-chat-hf",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\n\n'):
        town_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1]\
        .split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name']
        }
        towns[town_name] = town
    kingdom["towns"] = towns

In [ ]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: \
{town["description"]}')

## Generating Non-Player Characters (NPC's)

In [ ]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [ ]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
        ],
        temperature=1  #added to generate unique names
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [ ]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break

In [ ]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')

## Save the World
>Note: You will save your world state to a file different than the one shown in the video to allow future lessons to be consistent with the video. If later wish to build your own worlds, you will want to load your file rather than the saved file.

In [ ]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, '../shared_data/YourWorld_L1.json') #save to your version